In [50]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn import svm
import re
import nltk
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
from scipy.sparse import coo_matrix, hstack
from evaluation_submission import evaluate_task_a,evaluate_task_b
from sklearn.base import clone
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import spacy
from sklearn.metrics import f1_score

In [57]:
#ugly but functional function
def create_file(name,mis_labels,agr_labels):
    indexes=[x for x in range(5001,6001)]
    data={'id':indexes,
    'misogynous':mis_labels,
    'aggressiveness':agr_labels}
    dataframe=pd.DataFrame(data,index=indexes,columns=['id','misogynous','aggressiveness'])
    dataframe.to_csv(name,sep='\t',index=False)
def create_file_2(name,mis_labels,data_type):
    if data_type=='raw':
        indexes=[x for x in range(5001,6001)]
    else:
        indexes=[x for x in range (2014,3922)]
    data={'id':indexes,
    'misogynous':mis_labels}
    dataframe=pd.DataFrame(data,index=indexes,columns=['id','misogynous'])
    dataframe.to_csv(name,sep='\t',index=False)

In [58]:
def get_mean_embeddings(texts,embeddings):
    means = []
    dim = len(list(embeddings.values())[0])
    for text in texts :
        text = nltk.WordPunctTokenizer().tokenize(text)
        means.append(np.mean([embeddings[w] if w in embeddings else np.zeros(dim) for w in text], axis=0))
    return np.array(means)
def get_noun_phrases_embeddings(documents, lang='it_core_news_lg'):
    nlp = spacy.load(lang)
    phrased = []
    for idx, text in enumerate(documents):
        doc = nlp(text)
        all_vectors = [doc[start:end].vector for start,end,_ in noun_chunks(doc)]
        if not all_vectors:
            all_vectors = [np.zeros((300,))]
        phrases = np.mean(all_vectors, axis=0)
        phrased.append(phrases)
    return np.array(phrased)

def get_spacy_embeddings(documents, lang='it_core_news_lg'):
    nlp = spacy.load(lang)
    phrased = []
    for idx, text in enumerate(documents):
        doc = nlp(text)
        phrased.append(doc.vector)
    return np.array(phrased)
def run_cv(classifier, k_fold, data, labels, runs=10):
    accuracy_scores = []
    f1scores = []
    auc_scores = [0]
    confusion_matrices = []
    min_acc = 100
    labels = np.array(labels)
    skf = StratifiedKFold(k_fold)
    for run in range(0, runs):
        cv_splits = skf.split(data, labels)
        for train, test in cv_splits:
            traindata = data[train]
            y_traindata = labels[train]
            testdata = data[test]
            y_testdata = labels[test]
            model = clone(classifier)
            model.fit(traindata, y_traindata)
            result = model.predict(testdata)
            score = accuracy_score(y_testdata, result)
            accuracy_scores.append(score)
            f1sc = f1_score(y_testdata, result, average='weighted') 
            f1scores.append(f1sc)
            #auc = roc_auc_score(y_testdata, result, average='weighted', multi_class='ovr')
            #auc_scores.append(auc)
            if f1sc < min_acc:
                min_acc = f1sc 
                split_inidices = (train, test)
    print(np.mean(f1scores))
    print ("min cv F1: ", min_acc)   
    return (accuracy_scores, f1scores, auc_scores, split_inidices)

def real_score_A(predicted_mis,real_mis,predicted_agr,real_agr):
    f1sc_m = f1_score(real_mis, predicted_mis, average='macro')
    f1sc_a = f1_score(real_agr, predicted_agr, average='macro')  
    macro_score = (f1sc_m + f1sc_a) / 2
    print(f'macro score: {macro_score}')



In [59]:
#init
model = LogisticRegression(penalty='l2', dual=True, tol=0.0001, max_iter=100000,
                         C=3, fit_intercept=True, intercept_scaling=1.0, 
                         solver = 'liblinear', warm_start=False,
                         class_weight=None, random_state=None)
vectorizer = TfidfVectorizer(min_df=3,  max_features=None, 
    strip_accents='unicode', analyzer='word', token_pattern=r'\b[^\d\W]+\b',
    ngram_range=(1, 5),use_idf=True)


embeddings_index = {}
with open('glove/glove.twitter.27B.200d.txt', 'r', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
embeddings_glove=embeddings_index

In [60]:
#names=['tfidf','english_embeddings','italian_embeddings','tfidf + noun phrases','embeddings + noun phrases']
#names=[name+'.tsv' for name in names]

DATAFRAME_TRAIN_raw = pd.read_csv('AMI2020_TrainingSet/AMI2020_training_raw.tsv', sep='\t')
DATAFRAME_TRAIN_raw['clean']=DATAFRAME_TRAIN_raw['text']

DATAFRAME_TEST_raw=pd.read_csv('test/AMI2020_test_raw_gt.tsv',sep='\t')
DATAFRAME_TEST_raw['clean']=DATAFRAME_TEST_raw['text']

DATAFRAME_TRAIN_synt = pd.read_csv('AMI2020_TrainingSet/AMI2020_training_synt.tsv', sep='\t')
DATAFRAME_TRAIN_synt['clean']=DATAFRAME_TRAIN_synt['text']

DATAFRAME_TEST_synt=pd.read_csv('test/AMI2020_test_synt_gt.tsv',sep='\t')
DATAFRAME_TEST_synt['clean']=DATAFRAME_TEST_synt['text']


DATAFRAME_gold_raw = pd.read_csv('test/AMI2020_test_raw_gt.tsv', sep='\t')
DATAFRAME_gold_synt=pd.read_csv('test/AMI2020_test_synt_gt.tsv',sep='\t')


raw_real_mis,raw_real_agr=DATAFRAME_gold_raw['misogynous'],DATAFRAME_gold_raw['aggressiveness']
synt_real_mis=DATAFRAME_gold_synt['misogynous']

# TF IDF A si B

In [61]:
def results_tfidf_related(model,vectorizer,train_raw,test_raw,train_synt,test_synt,method_name='tfidf'):
    #task A
    RAW=pd.concat([train_raw,test_raw])
    SYNT=pd.concat([train_synt,test_synt])
    DATAFRAME=pd.concat([RAW,SYNT])
    TRAIN_DATAFRAME=pd.concat([train_raw,train_synt])
    TEST_DATAFRAME=pd.concat([test_raw,test_synt])

    X=vectorizer.fit_transform(RAW['clean'])

    #raw
    train_x,test_x=X[:5000],X[5000:]

    #misog
    model.fit(train_x,train_raw['misogynous'])
    result_mis=model.predict(test_x)

    #agr
    model.fit(train_x,train_raw['aggressiveness'])
    result_agr=model.predict(test_x)

    for i,(mis,agr) in enumerate(zip(result_mis,result_agr)):
        if mis==0 and agr==1:
            result_agr[i]=0 #Sunt 4 cazuri cand se intampla
    create_file('task A '+method_name+'.tsv',result_mis,result_agr)

    print(f'\ncv misogin for {method_name}')
    run_cv(model,10,train_x,train_raw['misogynous'])
    print(f'\ncv agresiv for {method_name}')
    run_cv(model,10,train_x,train_raw['aggressiveness'])
    print('\nreal score')
    real_score_A(result_mis,raw_real_mis,result_agr,raw_real_mis)
    '''
    #B
    print(DATAFRAME['clean'])
    X=vectorizer.fit_transform(DATAFRAME['clean'])
    x_train=vectorizer.transform(TRAIN_DATAFRAME['clean'])
    x_raw_test=vectorizer.transform(test_raw['clean'])
    x_synt_test=vectorizer.transform(test_synt['clean'])
    model.fit(x_train,TRAIN_DATAFRAME['misogynous'])

    print(x_train.shape)
    print(x_raw_test.shape)
    result_raw=model.predict(x_raw_test)
    create_file_2('task B raw '+method_name+'.tsv',result_raw,'raw')

    result_synt=model.predict(x_synt_test)
    create_file_2('task B synt '+method_name+'.tsv',result_synt,'synt')

    '''

In [62]:
def results_embedding_related(model,embeddings,train_raw,test_raw,train_synt,test_synt,method_name='embeddings'):
    #task A
    RAW=pd.concat([train_raw,test_raw])
    SYNT=pd.concat([train_synt,test_synt])
    DATAFRAME=pd.concat([RAW,SYNT])
    TRAIN_DATAFRAME=pd.concat([train_raw,train_synt])
    TEST_DATAFRAME=pd.concat([test_raw,test_synt])
    nlp=spacy.load('it_core_news_lg')
    if method_name=='glove':
        X=get_mean_embeddings(RAW['clean'].values,embeddings)
    else:
        #presupunand ca daca nu e glove simplu e spacy
        embeddingurile_train=[]
        for text in train_raw['clean'].values:
            embeddingurile_train.append(nlp(text).vector)

        embeddingurile_test=[]
        for text in test_raw['clean'].values:
            embeddingurile_test.append(nlp(text).vector)
        X=np.concatenate([embeddingurile_train,embeddingurile_test])    


    #raw
    train_x,test_x=X[:5000],X[5000:]

    #misog
    model.fit(train_x,train_raw['misogynous'])
    result_mis=model.predict(test_x)

    #agr
    model.fit(train_x,train_raw['aggressiveness'])
    result_agr=model.predict(test_x)

    for i,(mis,agr) in enumerate(zip(result_mis,result_agr)):
        if mis==0 and agr==1:
            result_agr[i]=0 #Sunt 4 cazuri cand se intampla

    create_file(method_name+' task A.tsv',result_mis,result_agr)

    print(f'\ncv misogin for{method_name}')
    run_cv(model,10,train_x,train_raw['misogynous'])
    print(f'\ncv agresiv for{method_name}')
    run_cv(model,10,train_x,train_raw['aggressiveness'])
    print('\nreal score')
    real_score_A(result_mis,raw_real_mis,result_agr,raw_real_mis)

    '''
    #B
    X=get_mean_embeddings(DATAFRAME['clean'],embeddings)
    x_train=get_mean_embeddings(TRAIN_DATAFRAME['clean'],embeddings)
    x_raw_test=get_mean_embeddings(test_raw['clean'],embeddings)
    x_synt_test=get_mean_embeddings(test_synt['clean'],embeddings)

    model.fit(x_train,TRAIN_DATAFRAME['misogynous'])

    result_raw=model.predict(x_raw_test)
    create_file_2(method_name+' task B raw.tsv',result_raw,'raw')

    result_synt=model.predict(x_synt_test)
    create_file_2(method_name+' task B synt.tsv',result_synt,'synt')
    '''


In [63]:
results_tfidf_related(model,vectorizer,DATAFRAME_TRAIN_raw,DATAFRAME_TEST_raw,DATAFRAME_TRAIN_synt,DATAFRAME_TEST_synt,'tfidf simplu')
results_embedding_related(model,embeddings_glove,DATAFRAME_TRAIN_raw,DATAFRAME_TEST_raw,DATAFRAME_TRAIN_synt,DATAFRAME_TEST_synt,'glove')
results_embedding_related(model,embeddings_glove,DATAFRAME_TRAIN_raw,DATAFRAME_TEST_raw,DATAFRAME_TRAIN_synt,DATAFRAME_TEST_synt,'spacy')


cv misogin for tfidf simplu
0.8827527300244864
min cv F1:  0.808

cv agresiv for tfidf simplu
0.8002388134462832
min cv F1:  0.5077334870538501

real score
macro score: 0.7064953843078551

cv misogin forglove
0.8183242384063837
min cv F1:  0.7622142694232932

cv agresiv forglove
0.7413811683837378
min cv F1:  0.48945024436519086

real score
macro score: 0.6906001893417045

cv misogin forspacy


KeyboardInterrupt: 